In [19]:
import pandas as pd
import numpy as np
import surprise
from surprise import Reader
from surprise import Dataset
from surprise import SVD
from surprise import accuracy
from surprise.prediction_algorithms import *
from surprise.model_selection import GridSearchCV
import pickle
pd.set_option('display.max_rows', None)
import streamlit

In [8]:

df =  pd.read_csv('./Data/clean_data2.csv')

In [10]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634964 entries, 0 to 634963
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  634964 non-null  int64  
 1   user        634964 non-null  object 
 2   item        634964 non-null  object 
 3   rating      634964 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 19.4+ MB


In [11]:

reader = Reader(line_format=u'rating user item', sep=',', rating_scale=(1, 5), skip_lines=1)

In [12]:

data = Dataset.load_from_df(df[["user", "item", "rating"]], reader)

In [13]:
train, test = surprise.model_selection.train_test_split(data, random_state=42)

In [9]:
model = SVD().fit(train)

In [10]:
surprise.accuracy.mae(model.test(test))

MAE:  0.4371


0.4370625514751679

In [13]:
surprise.accuracy.rmse(model.test(test))

RMSE: 0.5860


0.5859841295143399

In [15]:
param_grid = {'n_factors':[50,100,150],'n_epochs':[20,30],  'lr_all':[0.005,0.01],'reg_all':[0.02,0.1]}

In [16]:
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

In [18]:
params = gs.best_params['rmse']

In [19]:
pickle.dump( params, open( "./Data/save.p", "wb" ) )

In [14]:
svd_params = pd.read_pickle('./Data/save.p')

In [15]:
svd_params

{'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.1}

In [27]:
svdtuned = SVD(n_factors=svd_params['n_factors'], n_epochs=svd_params['n_epochs'],lr_all=svd_params['lr_all'], reg_all=svd_params['reg_all'])

In [29]:
svdtuned_model = svdtuned.fit(train)

In [30]:
surprise.accuracy.mae(svdtuned_model.test(test))

MAE:  0.4350


0.4350160466082846

In [31]:
surprise.accuracy.rmse(svdtuned_model.test(test))

RMSE: 0.5844


0.5844036922658483

In [25]:
param_grid2 = {'n_factors':[60,120,180],'n_epochs':[20,30,40],  'lr_all':[0.005,0.01,.1],'reg_all':[0.02,0.1]}

In [26]:
gs2 = GridSearchCV(SVD, param_grid2, measures=['rmse'], cv=5)
gs2.fit(data)

In [27]:
params2 = gs2.best_params['rmse']

In [29]:
#pickle.dump(params2, open ('./Data/save.p2', 'wb'))

In [16]:
svd_params2 = pd.read_pickle('./Data/save.p2')

In [17]:
svd_params2

{'n_factors': 180, 'n_epochs': 40, 'lr_all': 0.01, 'reg_all': 0.1}

In [22]:
svdtuned2 =  SVD(n_factors=svd_params2['n_factors'], n_epochs=svd_params2['n_epochs'],lr_all=svd_params2['lr_all'], reg_all=svd_params2['reg_all'])

In [20]:
svdtuned_model2 = svdtuned2.fit(train)


In [20]:
surprise.accuracy.mae(svdtuned_model2.test(test))

MAE:  0.4348


0.4347507261461443

In [21]:
surprise.accuracy.rmse(svdtuned_model2.test(test))

RMSE: 0.5838


0.5837716912798773

In [21]:
surprise.accuracy.fcp(svdtuned_model2.test(test))

FCP:  0.6747


0.6746622080077213

In [24]:
trainsetfull = data.build_full_trainset()

In [47]:
pickle.dump(final_model, open ('./Data/FM.sav', 'wb'))

In [48]:
lm = pickle.load(open('./Data/FM.sav', 'rb'))

In [25]:
final_model = svdtuned2.fit(trainsetfull)